# Naive

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import Naive
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/naive.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [8]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [9]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = Naive(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:12<00:00, 37.05it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 83.5000
	Loss_MB/train_phase/train_stream/Task000 = 1.1101
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 12.6608
100%|██████████| 469/469 [00:12<00:00, 37.99it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 46.9000
	Loss_MB/train_phase/train_stream/Task000 = 0.6848
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 12.3436
100%|██████████| 469/469 [00:11<00:00, 39.35it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 110.9000
	Loss_MB/train_phase/train_stream/Task000 = 0.5718
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 11.9175
100%|██████████| 469/469 [00:11<00:00, 39.31it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train_str

  5%|▌         | 1/20 [02:42<51:20, 162.12s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 43.2808
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x174f8f0a0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9114, 0.1148, 0.1265, 0.1234, 0.1478, 0.1127, 0.1183, 0.1046, 0.1159,
         0.1416, 0.0908, 0.0789, 0.1344, 0.0998, 0.1013, 0.0859, 0.0955, 0.1200,
         0.0975, 0.1187],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

 10%|█         | 2/20 [05:25<48:51, 162.88s/it]

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 54.2628
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17b14ac40>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9114, 0.1148, 0.1265, 0.1234, 0.1478, 0.1127, 0.1183, 0.1046, 0.1159,
         0.1416, 0.0908, 0.0789, 0.1344, 0.0998, 0.1013, 0.0859, 0.0955, 0.1200,
         0.0975, 0.1187],
        [0.8945, 0.9270, 0.1303, 0.1079, 0.1219, 0.0816, 0.0734, 0.1048, 0.1141,
         0.1719, 0.1035, 0.0848, 0.1288, 0.0915, 0.1125, 0.0766, 0.0874, 0.0981,
         0.1110, 0.0989],
        [0.8806, 0.8968, 0.9343, 0.0942, 0.1067, 0.1055, 0.0721, 0.1319, 0.1024,
         0.1536, 0.0947, 0.0727, 0.1280, 0.0804, 0.1609, 0.0653, 0.0590, 0.1129,
         0.1323, 0.0851],
        [0.8351, 0.8718, 0.9096, 0.9351, 0.0738, 0.0793, 0.0928, 0.1190, 0.0957,
         0.1422, 0.0827, 0.1071, 0.1818, 0.0864, 0.17

 40%|████      | 8/20 [26:28<42:51, 214.25s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.5278
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17fc4b9a0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9114, 0.1148, 0.1265, 0.1234, 0.1478, 0.1127, 0.1183, 0.1046, 0.1159,
         0.1416, 0.0908, 0.0789, 0.1344, 0.0998, 0.1013, 0.0859, 0.0955, 0.1200,
         0.0975, 0.1187],
        [0.8945, 0.9270, 0.1303, 0.1079, 0.1219, 0.0816, 0.0734, 0.1048, 0.1141,
         0.1719, 0.1035, 0.0848, 0.1288, 0.0915, 0.1125, 0.0766, 0.0874, 0.0981,
         0.1110, 0.0989],
        [0.8806, 0.8968, 0.9343, 0.0942, 0.1067, 0.1055, 0.0721, 0.1319, 0.1024,
         0.1536, 0.0947, 0.0727, 0.1280, 0.0804, 0.1609, 0.0653, 0.0590, 0.1129,
         0.1323, 0.0851],
        [0.8351, 0.8718, 0.9096, 0.9351, 0.0738, 0.0793, 0.0928, 0.1190, 0.0957,
         0.1422, 0.0827, 0.1071, 0.1818, 0.0864, 0.17

 45%|████▌     | 9/20 [29:41<38:03, 207.61s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 47.4515
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x28e5106d0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9114, 0.1148, 0.1265, 0.1234, 0.1478, 0.1127, 0.1183, 0.1046, 0.1159,
         0.1416, 0.0908, 0.0789, 0.1344, 0.0998, 0.1013, 0.0859, 0.0955, 0.1200,
         0.0975, 0.1187],
        [0.8945, 0.9270, 0.1303, 0.1079, 0.1219, 0.0816, 0.0734, 0.1048, 0.1141,
         0.1719, 0.1035, 0.0848, 0.1288, 0.0915, 0.1125, 0.0766, 0.0874, 0.0981,
         0.1110, 0.0989],
        [0.8806, 0.8968, 0.9343, 0.0942, 0.1067, 0.1055, 0.0721, 0.1319, 0.1024,
         0.1536, 0.0947, 0.0727, 0.1280, 0.0804, 0.1609, 0.0653, 0.0590, 0.1129,
         0.1323, 0.0851],
        [0.8351, 0.8718, 0.9096, 0.9351, 0.0738, 0.0793, 0.0928, 0.1190, 0.0957,
         0.1422, 0.0827, 0.1071, 0.1818, 0.0864, 0.17

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))